In [9]:
import os
import sys
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import umap
from sklearn.feature_selection import SelectKBest, f_classif
from matplotlib.patches import Ellipse
sys.path.append("../../Utils")
from loaders import HNSCCFeatureHandler

# Setup
METADATA_PATH = "../../Supplementary_Tables/ST1/RAW_HNSCC_METADATA_NEW.csv"
VALID_IDS_PATH = "../../Utils/Lists/cv_ids.txt"
HOLD_IDS_PATH = '../../Utils/Lists/holdout_ids.txt'

# Common helper

def plot_svd_variance(S, svd_cutoff, title, outfile):
    prop_var_explained = (S**2) / np.sum(S**2)
    plt.figure(figsize=(6, 3), dpi=300)

    plt.subplot(1, 2, 1)
    plt.plot(S[:50], 'o', markersize=0.75)
    plt.xlabel("SVD #")
    plt.ylabel("Singular Value")
    plt.axvline(x=svd_cutoff, color='red', linestyle=(0, (3, 1, 1, 1)), linewidth=0.5)

    plt.subplot(1, 2, 2)
    plt.plot(prop_var_explained[:50], 'o', markersize=0.75)
    plt.xlabel("SVD #")
    plt.ylabel("Prop. of Variance Explained")
    plt.axvline(x=svd_cutoff, color='red', linestyle=(0, (3, 1, 1, 1)), linewidth=0.5)

    plt.tight_layout()
    plt.savefig(outfile)
    plt.close()

def confidence_ellipse(x, y, ax, n_std=2.0, facecolor='none', **kwargs):
    if x.size <= 1:
        return
    cov = np.cov(x, y)
    if np.any(np.isnan(cov)) or np.linalg.det(cov) == 0:
        return
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]
    width, height = 2 * n_std * np.sqrt(vals)
    angle = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    ellipse = Ellipse((np.mean(x), np.mean(y)), width, height, angle=angle, facecolor=facecolor, **kwargs)
    ax.add_patch(ellipse)

def plot_umap(U, y, title, outfile, k=6):
    embedding = umap.UMAP(n_components=2, n_neighbors=50, min_dist=0.5, metric='cosine', random_state=42).fit_transform(U[:, :k])
    umap_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'])
    umap_df['Treatment Response'] = y.reset_index(drop=True)
    colors = sns.color_palette('colorblind', 10)
    selected_colors = [colors[5], colors[0]]
    color_map = {t: selected_colors[i % 2] for i, t in enumerate(umap_df['Treatment Response'].unique())}

    fig, ax = plt.subplots(figsize=(4.2, 4.2), dpi=400)
    for t, color in reversed(list(color_map.items())):
        subset = umap_df[umap_df['Treatment Response'] == t]
        ax.scatter(subset['UMAP1'], subset['UMAP2'], color=color, label=t, s=10, linewidth=0.2, alpha=0.8)
        confidence_ellipse(subset['UMAP1'].values, subset['UMAP2'].values, ax, n_std=2, facecolor=color, alpha=0.2, edgecolor='none')

    ax.set_xlabel('UMAP 1', fontsize=12)
    ax.set_ylabel('UMAP 2', fontsize=12)
    ax.legend(loc='upper right', fontsize=8, frameon=True, edgecolor='black')
    sns.despine()
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outfile, dpi=600)
    plt.close()

# Analysis blocks

features = {
    'TUMORFRAC': ("../../Data", 'tumorfrac', 1, 1, 1, False),
    'rMDS': ("*.hg38.frag.interval_mds.tsv", 'rmds', 0, 4, 6, True),
    'DELFI': ("*.hg38.frag.delfi.bed", 'delfi', 1, 12, 6, True),
    'Fragment Length': ("*.hg38.frag.frag_length_intervals.bed", 'fraglen', 1, 4, 5, True),
    'End Motif Proportion': ("*.hg38.frag.end_motifs.tsv", 'endmotifs', 0, 3, 5, True),
    'GC% Corrected Coverage': ("*.hg38.frag.delfi.bed", 'coverage', 1, 11, 7, True),
    'Copy Number logR (ichorCNA)': ("*.cna.seg", 'cna', 1, 5, 6, True),
}

for label, (pattern, tag, skip_rows, col_info, svd_k, do_batch) in features.items():
    print(label)
    hc_data = HNSCCFeatureHandler(METADATA_PATH, VALID_IDS_PATH, HOLD_IDS_PATH)
    if tag == 'tumorfrac':
        with open(VALID_IDS_PATH) as f:
            valid_ids = [line.strip() for line in f]
        values = {}
        for id in valid_ids:
            try:
                with open(f"{pattern}/{id}/{id}.params.txt") as file:
                    lines = file.readlines()
                    values[id] = float(lines[1].split('\t')[1])
            except Exception:
                continue
        df = pd.DataFrame(list(values.items()), columns=['ID', 'Tumor Fraction']).set_index('ID')
        meta = pd.read_csv(METADATA_PATH).set_index('ID')
        df = df.join(meta['Treatment Response']).dropna()
        X = df[['Tumor Fraction']]
        y = df['Treatment Response']
    else:
        if tag!='cna':
            hc_data.load_feature_to_dataframe(f"../../Data/{pattern}", skip_rows, col_info)
        else:
            hc_data.load_feature_to_dataframe(f"../../Data/{pattern}", skip_rows, col_info, na_value='NA')
        if tag=='coverage':
            hc_data.normalize_total_sum()
        hc_data.normalize_zscore()
        hc_data.merge_feature_metadata()
        if do_batch:
            hc_data.batch_correct()
        df = hc_data.get_raw_features()
        y = hc_data.get_metadata_col("Treatment Response")
        X = pd.concat([df, y], axis=1)
        y = X.pop("Treatment Response")

    U, S, _ = np.linalg.svd(X, full_matrices=False)
    plot_svd_variance(S, svd_k-1, f"{label} SVD", f"../SF5/SF5.{tag.upper()}.SVD.pdf")
    plot_umap(U, y, label, f"SF4.{tag.upper()}.pdf", k=svd_k)


TUMORFRAC
rMDS
DELFI
Fragment Length
End Motif Proportion
GC% Corrected Coverage
Copy Number logR (ichorCNA)
